In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import glob
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
from skimage import io
from skimage import transform
import tensorflow as tf
import utils

%matplotlib inline

In [ ]:
map_characters = utils.load_characters()

In [ ]:
for k, v in map_characters.items():
    print("{} -> {}".format(k, v))

In [ ]:
pics, labels = utils.load_pictures(map_characters)

In [ ]:
print("Images shape:")
print(pics.shape)

In [ ]:
utils.show_random(pics, labels, map_characters)

## Splitting into Train / Validation

In [ ]:
pics_train, labels_train, pics_val, labels_val = utils.split(pics, labels, p=0.95)

In [ ]:
print("Training data:")
print("X: {}".format(pics_train.shape))
print("Y: {}".format(labels_train.shape))
print()
print("Validation data:")
print("X: {}".format(pics_val.shape))
print("Y: {}".format(labels_val.shape))

## Defining model

In [ ]:
H, W, C = pics[0].shape
NUM_CLASSES = len(map_characters)

In [ ]:
def load_architecture():
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.uint8, shape=[None, H, W, 3], name="x")
    y = tf.placeholder(tf.uint8, shape=[None, NUM_CLASSES], name="y")
    
    dropout_rate = tf.placeholder_with_default(0.3, shape=(), name="dropout_rate")
    
    is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
    
    init = tf.contrib.layers.xavier_initializer()
    
    out = tf.divide(x, 255)
    
    out = tf.layers.conv2d(out, filters=16, kernel_size=[3,3], activation=tf.nn.elu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.layers.conv2d(out, filters=32, kernel_size=[3,3], activation=tf.nn.elu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.layers.conv2d(out, filters=64, kernel_size=[3,3], activation=tf.nn.elu, kernel_initializer=init, padding="same")
    out = tf.layers.max_pooling2d(out, pool_size=(2, 2), strides=[2,2])
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.contrib.layers.flatten(out)
    
    out = tf.layers.dense(out, units=512, activation=tf.nn.relu, kernel_initializer=init)
    
    out = tf.layers.dropout(out, rate=dropout_rate, training=is_training)
    
    out = tf.layers.dense(out, units=NUM_CLASSES, kernel_initializer=init)
    
    return x, y, is_training, dropout_rate, out

In [ ]:
def load_loss(y, out):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out, name="mean_loss")
    loss = tf.reduce_mean(loss, name="loss")
    return loss

In [ ]:
def load_accuracy(y, out):
    pred = tf.argmax(out, axis=-1)
    gt = tf.argmax(y, axis=-1)
    
    matches = tf.equal(pred, gt)
    
    return tf.reduce_mean(tf.cast(matches, tf.float32), name="acc")

In [ ]:
def load_trainer(loss):
    opt = tf.train.AdamOptimizer()
    return opt.minimize(loss)

In [ ]:
def register_scalars(m):
    for k, v in m.items():
        tf.summary.scalar(k, v)

In [ ]:
def register_images(m):
    for k, v in m.items():
        tf.summary.image(k, v)

In [ ]:
def trainable_parameters():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    return total_parameters

In [ ]:
def load_model():
    x, y, is_training, dropout_rate, out = load_architecture()
    loss = load_loss(y, out)
    acc = load_accuracy(y, out)
    upd = load_trainer(loss)
    
    register_scalars({"info_loss": loss, "info_acc": acc})
    register_images({"input": x})

    info = tf.summary.merge_all()
    
    return x, y, is_training, dropout_rate, out, loss, acc, upd, info

# Train model

In [ ]:
def load_session():
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    return sess

In [ ]:
def train(sess, model, pics_train, labels_train, pics_val, labels_val, epochs, batch_size, train_writer, val_writer, use_dropout=False):
    N, _, _, _ = pics_train.shape
    idxs = np.arange(N)
    
    x, y, is_training, dropout_rate, out, loss, acc, upd, info = model
    
    d_rate = 0.3 if use_dropout else 0.
    
    i=0

    for ep in tqdm(range(epochs)):
        np.random.shuffle(idxs)
        pics_train = pics_train[idxs]
        labels_train = labels_train[idxs]

        for b in range(0, N, batch_size):
            X_batch = pics_train[b:b+batch_size]
            Y_batch = labels_train[b:b+batch_size]

            if X_batch.shape[0] < BATCH_SIZE:
                break

            graph_info, _ = sess.run([info, upd], feed_dict={x: X_batch, y: Y_batch, is_training: True, dropout_rate: d_rate})
            train_writer.add_summary(graph_info, i)
            
            graph_info, = sess.run([info], feed_dict={x: pics_val, y: labels_val, is_training: False, dropout_rate: d_rate})
            val_writer.add_summary(graph_info, i)
            
            i+=1

In [ ]:
def predict(img):
    img_batch = np.reshape(img, [1, H, W, 3])
    graph_out, = sess.run([out], feed_dict={x: img_batch})
    char = np.argmax(np.squeeze(graph_out))
    plt.imshow(img)
    plt.title(map_characters[char])
    plt.show()

### Overfitting data first

In [ ]:
pics_train_sm, labels_train_sm = utils.get_small_dataset(pics_train, labels_train, p=0.01)

In [ ]:
print("New training dataset size: {}".format(pics_train_sm.shape[0]))

In [ ]:
model = load_model()
print("Trainable parameters: {}".format(trainable_parameters()))

In [ ]:
EPOCHS = 20
BATCH_SIZE = 32
LOGS_DIR = "logs"

sess = load_session()

t_writer = tf.summary.FileWriter(os.path.join(LOGS_DIR, "overfit", "train"), graph=sess.graph)
v_writer = tf.summary.FileWriter(os.path.join(LOGS_DIR, "overfit", "val"), graph=sess.graph)

train(sess, model, pics_train_sm, labels_train_sm, pics_val, labels_val, EPOCHS, BATCH_SIZE, t_writer, v_writer)

In [ ]:
idx = np.random.choice(pics_train.shape[0])
predict(pics_train[idx])

In [ ]:
idx = np.random.choice(pics_val.shape[0])
predict(pics_val[idx])